In [1]:
import json
import calculations as calc

In [2]:
# read sample_events.json and sample_input.json
folder_name = "FICC-code_to_connect"

event_path = f'{folder_name}/sample_events.json'
input_path = f'{folder_name}/sample_input.json'
with open(event_path, 'r') as f:
    events = json.load(f)
with open(input_path, 'r') as f:
    inputs = json.load(f)

In [3]:
def convert_list_to_dict(data):
    result = {}
    for d in data:
        event_id = d['EventId']
        if event_id not in result:
            result[event_id] = []
        result[event_id].append(d)
    return result

events_dict = convert_list_to_dict(events)
events_dict

{1: [{'EventId': 1,
   'EventType': 'ConfigEvent',
   'm': 0.01,
   'b': 0.07,
   'DivisorRatio': 300000,
   'Spread': 2}],
 2: [{'EventId': 2,
   'EventType': 'ConfigEvent',
   'm': 0.03,
   'b': 0.08,
   'DivisorRatio': 300000,
   'Spread': 1}],
 3: [{'EventId': 3,
   'EventType': 'TradeEvent',
   'Ccy': 'CHX',
   'BuySell': 'sell',
   'Tenor': '1M',
   'Quantity': 3000,
   'TradeId': 'T1'}],
 4: [{'EventId': 4,
   'EventType': 'TradeEvent',
   'Ccy': 'JPX',
   'BuySell': 'buy',
   'Tenor': '1M',
   'Quantity': 1000,
   'TradeId': 'T2'}],
 5: [{'EventId': 5,
   'EventType': 'ConfigEvent',
   'm': 0.02,
   'b': 0.08,
   'DivisorRatio': 100000,
   'Spread': 3}],
 6: [{'EventId': 6, 'EventType': 'FXMidEvent', 'Ccy': 'CHX', 'rate': 0.88}],
 7: [{'EventId': 7, 'EventType': 'FXMidEvent', 'Ccy': 'KRX', 'rate': 1322.06}],
 8: [{'EventId': 8,
   'EventType': 'TradeEvent',
   'Ccy': 'KRX',
   'BuySell': 'buy',
   'Tenor': '6M',
   'Quantity': 2000,
   'TradeId': 'T3'}],
 9: [{'EventId': 9,
   

In [21]:
def process_one_json_input(first_input):
    event_id = first_input['EventId']
    temp_event_list = []
    all_event_types = set(("TradeEvent", "FXMidEvent", "ConfigEvent"))
    for i in range(1, event_id+1):
        temp_event_list.extend(events_dict[i])

    # check if all event types are present
    for event in temp_event_list:
        event_type = event['EventType']
        if event_type in all_event_types:
            all_event_types.remove(event_type)

    quote_status = ""
    if len(all_event_types) != 0:
        quote_status = "EXCEPTION"
    
    # check if TradeEvent is present in all_event_types
    trade_exist = False
    if "TradeEvent" not in all_event_types:
        net_quantity = "NA"
        quote_status = "EXCEPTION"
    else:
        net_quantity = 0
        for one_event in temp_event_list:
            if one_event['EventType'] == 'TradeEvent':
                if one_event['Ccy'] == first_input['Ccy'] and one_event['Tenor'] == first_input['Tenor']:
                    trade_exist = True
                    net_quantity += calc.change_quantity(one_event)

    small_output_json = {
        "EventId": None, 
        "Ccy": None,
        "Tenor": None,
        "Position": "NA",
        "Bid": "NA",
        "Ask": "NA",
        "QuoteStatus": None
    }

    small_output_json['EventId'] = first_input['EventId']
    small_output_json['Ccy'] = first_input['Ccy']
    small_output_json['Tenor'] = first_input['Tenor']
    small_output_json['Position'] = net_quantity if trade_exist else "NA"
    small_output_json['QuoteStatus'] = quote_status if trade_exist else "EXCEPTION"

    return small_output_json

In [22]:
process_one_json_input(inputs[2])

{'EventId': 20,
 'Ccy': 'GBX',
 'Tenor': '1M',
 'Position': 'NA',
 'Bid': 'NA',
 'Ask': 'NA',
 'QuoteStatus': 'EXCEPTION'}

In [4]:
first_input = inputs[0]
first_input

{'Ccy': 'CHX', 'Tenor': '1M', 'EventId': 5}

In [5]:
event_id = first_input['EventId']
temp_event_list = []
all_event_types = set(("TradeEvent", "FXMidEvent", "ConfigEvent"))
for i in range(1, event_id+1):
    temp_event_list.extend(events_dict[i])

# check if all event types are present
for event in temp_event_list:
    event_type = event['EventType']
    if event_type in all_event_types:
        all_event_types.remove(event_type)

quote_status = ""
if len(all_event_types) != 0:
    quote_status = "EXCEPTION"
print(quote_status)

EXCEPTION


In [6]:
# for calculating net quantity we need to consider only TradeEvents
# must be same currency and tenor
net_quantity = 0
for one_event in temp_event_list:
    if one_event['EventType'] == 'TradeEvent':
        if one_event['Ccy'] == first_input['Ccy'] and one_event['Tenor'] == first_input['Tenor']:
            net_quantity += calc.change_quantity(one_event)

net_quantity

-3000

In [7]:
small_output_json = {
    "EventId": None, 
    "Ccy": None,
    "Tenor": None,
    "Position": None,
    "Bid": None,
    "Ask": None,
    "QuoteStatus": None
}

small_output_json['EventId'] = first_input['EventId']
small_output_json['Ccy'] = first_input['Ccy']
small_output_json['Tenor'] = first_input['Tenor']
small_output_json['Position'] = net_quantity
small_output_json['QuoteStatus'] = quote_status

small_output_json

{'EventId': 5,
 'Ccy': 'CHX',
 'Tenor': '1M',
 'Position': -3000,
 'Bid': None,
 'Ask': None,
 'QuoteStatus': 'EXCEPTION'}